In [1]:
import pandas as pd
import numpy as np
import scipy as sp

pd.set_option("display.max_columns", None)

data_missing_less_40p = pd.read_csv('Assignment_1_data_missing_less_40p.csv', index_col=False)

noncontinuous_var = ['gender', 'sofa_coagulation', 'sofa_cardiovascular','sofa_cns', 'sofa_renal', 'charlson_comorbidity_index']

data_continuous = data_missing_less_40p.loc[:, ~data_missing_less_40p.columns.isin(noncontinuous_var)]
data_continuous.to_csv('data_continuous.csv', index=False)  #comment to not write data


In [45]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.impute import KNNImputer, SimpleImputer

data_continuous = pd.read_csv('data_continuous.csv', index_col=False)
noncontinuous_encoded = pd.read_csv('noncontinuous_encoded.csv', index_col=False)

def make_knn_imputer(table, name_select_first_col, name_select_remaining_col):
    knn_imputer = KNNImputer()
    
    X = table[name_select_first_col].to_numpy()
    X = X.reshape(-1, 1)    
   
    knn_imputed_init = pd.DataFrame(knn_imputer.fit_transform(X), columns = [name_select_first_col])
    
    for i in name_select_remaining_col:
        X = lambda i: table[i].to_numpy()
        X = X(i).reshape(-1, 1)
        
        knn_imputed_temp = pd.DataFrame(knn_imputer.fit_transform(X), columns = [i])
        knn_imputed_init = pd.concat([knn_imputed_init, knn_imputed_temp], axis=1)
    
    output = knn_imputed_init
    
    return output

knn_imputed_contin = make_knn_imputer(data_continuous, 'age', list(data_continuous)[1:-1])
knn_imputed_contin = pd.concat([knn_imputed_contin, noncontinuous_encoded[['gender', 'outcome']]], axis=1)
knn_imputed_contin.to_csv('knn_imputed_contin.csv', index=False)  #comment to not write data

#def addFeatures(feature_table, target_table):
#    return pd.concat([target_table, feature_table], axis=1)

def make_imp_median(table, name_select_first_col, name_select_remaining_col):
    imp_median = SimpleImputer(strategy='median')
    
    X = table[name_select_first_col].to_numpy()
    X = X.reshape(-1, 1)    
   
    imp_median_init = pd.DataFrame(imp_median.fit_transform(X), columns = [name_select_first_col])
    
    for i in name_select_remaining_col:
        X = lambda i: table[i].to_numpy()
        X = X(i).reshape(-1, 1)
        
        imp_median_temp = pd.DataFrame(imp_median.fit_transform(X), columns = [i])
        imp_median_init = pd.concat([imp_median_init, imp_median_temp], axis=1)
    
    output = imp_median_init
    
    return output

median_imputed_contin = make_imp_median(data_continuous, 'age', list(data_continuous)[1:-1])
median_imputed_contin = pd.concat([median_imputed_contin, noncontinuous_encoded[['gender', 'outcome']]], axis=1)
median_imputed_contin.to_csv('median_imputed_contin.csv', index=False)  #comment to not write data


Kolmogorov-Smirnov test for goodness of fit to normality

In [46]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy.stats import kstest, norm

data_continuous = pd.read_csv('data_continuous.csv', index_col=False)
data_continuous = data_continuous.drop('outcome',axis= 1)

for i in list(data_continuous):
    print(i, kstest(data_continuous[i].dropna(), norm.cdf)) #.dropna() MNAR

age KstestResult(statistic=1.0, pvalue=0.0)
heart_rate_min KstestResult(statistic=1.0, pvalue=0.0)
heart_rate_max KstestResult(statistic=1.0, pvalue=0.0)
heart_rate_mean KstestResult(statistic=1.0, pvalue=0.0)
mbp_min KstestResult(statistic=0.9983749659384641, pvalue=0.0)
mbp_max KstestResult(statistic=1.0, pvalue=0.0)
mbp_mean KstestResult(statistic=1.0, pvalue=0.0)
sbp_min KstestResult(statistic=0.9999449677831892, pvalue=0.0)
sbp_max KstestResult(statistic=1.0, pvalue=0.0)
sbp_mean KstestResult(statistic=1.0, pvalue=0.0)
dbp_min KstestResult(statistic=0.9999174486104809, pvalue=0.0)
dbp_max KstestResult(statistic=1.0, pvalue=0.0)
dbp_mean KstestResult(statistic=1.0, pvalue=0.0)
temperature_min KstestResult(statistic=1.0, pvalue=0.0)
temperature_max KstestResult(statistic=1.0, pvalue=0.0)
temperature_mean KstestResult(statistic=1.0, pvalue=0.0)
glucose_min KstestResult(statistic=0.9999999999987201, pvalue=0.0)
glucose_max KstestResult(statistic=0.9999999999987201, pvalue=0.0)
wbc_min

smoten on the imputed

In [50]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier, SGDRegressor, RidgeCV, LassoCV
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from imblearn.over_sampling import SMOTEN

knn_imputed_contin = pd.read_csv('knn_imputed_contin.csv', index_col=False)
median_imputed_contin = pd.read_csv('median_imputed_contin.csv', index_col=False)

pd.set_option("display.max_columns", None)

def smotenThis(table, lst, y_var):    
    X = table[lst]
    y = table[y_var].to_numpy()
    y = y.ravel()
    y = LabelEncoder().fit_transform(y)

    oversample = SMOTEN()
    X, y = oversample.fit_resample(X, y)    
    array_X = pd.DataFrame(X)
    array_y = pd.DataFrame(y, columns = [y_var])
    
    output = pd.concat([array_X, array_y], axis=1)
    
    return output

smoten_knn_contin = smotenThis(knn_imputed_contin, list(knn_imputed_contin.drop('outcome',axis= 1)), 'outcome')
smoten_knn_contin.to_csv('smoten_knn_contin.csv', index=False)  #comment to not write data

smoten_median_imputed_contin = smotenThis(median_imputed_contin, list(median_imputed_contin.drop('outcome',axis= 1)), 'outcome')
smoten_median_imputed_contin.to_csv('smoten_median_imputed_contin.csv', index=False)  #comment to not write data


    
